In [ ]:
!pip install tensorflow-text

     |████████████████████████████████| 4.3MB 5.3MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import models,layers
import tensorflow_text as text

In [ ]:
text_input = layers.Input(shape=(),dtype=tf.string)
prep = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = prep(text_input)

encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/2",trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"] #[batchsize,512]
sequence_ouptut = outputs["sequence_output"] #[batchsize,seqlength,512]

In [ ]:
embedding_model = tf.keras.Model(text_input, pooled_output)
sentences = tf.constant(["(your text here)"])
print(embedding_model(sentences).shape)

(1, 512)


In [ ]:
embedding_model2 = tf.keras.Model(text_input, sequence_ouptut)
sentences = tf.constant(["My name is anshu here, who are you"])
print(embedding_model2(sentences).shape)

(1, 128, 512)


In [ ]:
sentences.shape

TensorShape([1])

In [ ]:
doc1 = ["India"]
doc2 = ["I love coding"]
doc3 = ["I love python porgamming and Data Science. Machine Learning is amazing."]
data = [doc1,doc2,doc3]
for doc in data:
  doc = tf.constant(doc)
  print(embedding_model(doc).shape)

(1, 512)
(1, 512)
(1, 512)


In [ ]:
for doc in data:
  doc = tf.constant(doc)
  print(embedding_model2(doc).shape)

(1, 128, 512)
(1, 128, 512)
(1, 128, 512)


# Text Classification

In [ ]:
# Text Classification

In [ ]:
!wget -q https://www.dropbox.com/s/p1z32pkhs9j7cv3/test_data.txt
!wget -q https://www.dropbox.com/s/ed3qcu1231e8ubx/train_5500.txt

In [ ]:
train_data = open("train_5500.txt").readlines()
test_data = open("test_data.txt").readlines()

In [ ]:
len(train_data)

5452

In [ ]:
len(test_data)

500

In [ ]:
train_data[:5]

['DESC:manner How did serfdom develop in and then leave Russia ?\n',
 'ENTY:cremat What films featured the character Popeye Doyle ?\n',
 "DESC:manner How can I find a list of celebrities ' real names ?\n",
 'ENTY:animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?\n',
 'ABBR:exp What is the full form of .com ?\n']

In [ ]:
def prepare_data(data):
  x = []
  y = []
  for doc in data:
    docs = doc.split(" ")
    y.append(docs[0].split(":")[0])
    x.append(" ".join(docs[1:]).strip())
  return x,y


In [ ]:
xtrain,ytrain = prepare_data(train_data)
xtest,ytest = prepare_data(test_data)

In [ ]:
import pandas as pd

In [ ]:
ytrain = pd.DataFrame(ytrain)
ytrain = pd.get_dummies(ytrain)
ytrain.head()

,0_ABBR,0_DESC,0_ENTY,0_HUM,0_LOC,0_NUM
0,0,1,0,0,0,0
1,0,0,1,0,0,0
2,0,1,0,0,0,0
3,0,0,1,0,0,0
4,1,0,0,0,0,0


In [ ]:
ytest = pd.DataFrame(ytest)
ytest = pd.get_dummies(ytest)
ytest.head()

,0_ABBR,0_DESC,0_ENTY,0_HUM,0_LOC,0_NUM
0,0,0,0,0,0,1
1,0,0,0,0,1,0
2,0,0,0,1,0,0
3,0,1,0,0,0,0
4,0,0,0,0,0,1


In [ ]:
categories = list(ytrain.columns)
categories = [i[2:] for i in categories]
categories

['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']

### Text Classification using Transfer learning

In [ ]:
# create a function which will accept a text doc and return the corresponding vector
def sent2vec(doc):
  vector = embedding_model(tf.squeeze(tf.cast(doc,tf.string),axis=1))
  return vector

In [ ]:
from tensorflow.keras import models,layers

In [ ]:
input_layer = layers.Input(shape=(1,),dtype='string')
embedding_layer = layers.Lambda(sent2vec,output_shape=(512,))(input_layer)
dense1 = layers.Dense(256,activation='relu')(embedding_layer)
output_layer = layers.Dense(6,activation='softmax')(dense1)

model = models.Model(inputs=[input_layer],outputs=output_layer)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'word_embeddings/embeddings:0' shape=(30522, 512) dtype=float32>
  <tf.Variable 'position_embedding/embeddings:0' shape=(512, 512) dtype=float32>
  <tf.Variable 'type_embeddings/embeddings:0' shape=(2, 512) dtype=float32>
  <tf.Variable 'embeddings/layer_norm/gamma:0' shape=(512,) dtype=float32>
  <tf.Variable 'embeddings/layer_norm/beta:0' shape=(512,) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/query/kernel:0' shape=(512, 8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/query/bias:0' shape=(8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/key/kernel:0' shape=(512, 8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/key/bias:0' shape=(8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/value/kernel:0' shape=(512, 8, 64) dtype=float32>
  <tf.Variable 't

The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'word_embeddings/embeddings:0' shape=(30522, 512) dtype=float32>
  <tf.Variable 'position_embedding/embeddings:0' shape=(512, 512) dtype=float32>
  <tf.Variable 'type_embeddings/embeddings:0' shape=(2, 512) dtype=float32>
  <tf.Variable 'embeddings/layer_norm/gamma:0' shape=(512,) dtype=float32>
  <tf.Variable 'embeddings/layer_norm/beta:0' shape=(512,) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/query/kernel:0' shape=(512, 8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/query/bias:0' shape=(8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/key/kernel:0' shape=(512, 8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/key/bias:0' shape=(8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/value/kernel:0' shape=(512, 8, 64) dtype=float32>
  <tf.Variable 't

In [ ]:
import numpy as np
xtrain = np.array(xtrain).reshape(-1,1)
xtest = np.array(xtest).reshape(-1,1)
print(xtrain.shape)
print(xtest.shape)

(5452, 1)
(500, 1)


In [ ]:
model.fit(xtrain,ytrain,epochs=4,batch_size=64,validation_data=(xtest,ytest))

Epoch 1/4
86/86 [==============================] - 465s 5s/step - loss: 1.0941 - accuracy: 0.5816 - val_loss: 0.8676 - val_accuracy: 0.6620
Epoch 2/4
86/86 [==============================] - 463s 5s/step - loss: 0.9886 - accuracy: 0.6212 - val_loss: 0.7115 - val_accuracy: 0.7800
Epoch 3/4
86/86 [==============================] - 464s 5s/step - loss: 0.9388 - accuracy: 0.6399 - val_loss: 0.6178 - val_accuracy: 0.7640
Epoch 4/4
86/86 [==============================] - 464s 5s/step - loss: 0.9144 - accuracy: 0.6517 - val_loss: 0.6001 - val_accuracy: 0.7740


In [ ]:
new_doc = ["Who is president of United States of America?","What is price for Harley Davidson?"]
new_doc = np.array(new_doc).reshape(-1,1)
output = model.predict(new_doc)

In [ ]:
output2 = [np.argmax(i) for i in output]
output2 = [categories[i] for i in output2]
output2

['HUM', 'DESC']